In [ ]:
import re
import pandas as pd

In [ ]:
!pip install sqlalchemy

In [ ]:

def extract_attribute(text):
    lines = text.split('\n')
    un = 'Not found'
    def un_find(text):
      un = 'Not found'
      lines = text.split('\n')
      for line in lines:
        if 'UN number' in line:
          if ':' in line:
            un = line.split(':')[1].strip()
          else:
            index = lines.index(line)+1
            if ':' in lines[index]:
              un = lines[index].split(':')[1].strip()
            else:
              un = lines[index].strip()
      return un

    # Split the text into lines

    # Initialize variable to store product name
    product_name = "Not found"
    company_name = 'Not found'
    chemical_name = 'Not found'
    substance_use = 'Not found'
    cas = []
    ec = []
    hazard = ''
    hazard_lines = []
    in_hazard_section = False
    un = un_find(text)

    # Search for the line containing "Product name"
    for line in lines:
        if "Product name" in line or 'Trade name' in line:
            if len(re.findall('Trade',line)) ==1:
              if ':' in line:
                product_name = line.split(":")[1].strip()
              else:
                index = lines.index(line)+1
                if ':' in lines[index]:
                  product_name = lines[index].split(":")[1].strip()
            # Extract the product name from the line
            if len(re.findall('Product',line)) ==1:
              if ':' in line:
                product_name = line.split(":")[1].strip()
        if "Company name of supplier" in line or 'Company ' in line:
          if ':' in line:
            company_name = line.split(":")[1].strip()
          else:
            index = lines.index(line)+1
            if ':' in lines[index]:
              company_name = lines[index].split(':')[1].strip()
        if 'Supplier :' in line:
            index = lines.index(line)+1
            company_name = lines[index].strip()
        if 'Substance name' in line or 'Chemical name' in line or 'Chemical Name' in line:
          if ':' in line:
            chemical_name = line.split(":")[1].strip()
          else:
            index = lines.index(line)+1
            if ':' in lines[index]:
              chemical_name = lines[index].split(':')[1].strip()
        if 'Recommended use' in line or 'Specific end use(s)' in line:
            if ':' in line:
              substance_use = line.split(":", 1)[1].strip()
        if "Use of the" in line :
            if ":" in line:
                substance_use = line.split(":", 1)[1].strip()
            else:
                index = lines.index(line) + 1
                if index < len(lines):
                    line = lines[index].strip()
                    if ":" in line:
                        substance_use = line.split(":", 1)[1].strip()
                    else:
                      next_index = index+1
                      if next_index<len(lines):
                        next_line = lines[next_index].strip()
                        substance_use = next_line.split(":", 1)[1].strip()
        if "SECTION 2" in line:
            in_hazard_section = True
        else:
          if "SECTION 3" in line:
            in_hazard_section = False
          else:
            if in_hazard_section == True:
              if 'Category' in line:
                hazard = 1
                break
              else:
                hazard = 0



    cas = list(set(re.findall(r'\d+ -\d{2}-\d\b',text) or re.findall(r'\d+-\d{2}-\d\b', text)))
    ec = list(set(re.findall(r'\d+ -\d{3}-\d\b',text) or re.findall(r'\d+-\d{3}-\d\b',text)))


    return product_name, company_name, chemical_name, substance_use, cas, ec, hazard, un

In [ ]:
columns = []
df= pd.DataFrame(columns=columns)
def text_to_df(text):
  global df
  product_name, company_name, chemical_name, substance_use, cas, ec, hazard, un = extract_attribute(text)
  data = {
            'Product_name': [product_name],
            'Company_name': [company_name],
            'Chemical':[chemical_name],
            'Market_segmentation': [substance_use],
            'CAS_No': [", ".join(cas)],
            'EC_No': [", ".join(ec)],
            'Hazard': [hazard],
            'UN_number': [un]
    }
  df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
  return df



In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.3 MB/s eta 0:00:00


In [ ]:
from pypdf import PdfReader
pdf_files = [''] #import the pdf
text_list=[]
for pdf in pdf_files:
  reader = PdfReader(pdf)
  text = ""
  for page in reader.pages:
      text += page.extract_text()

  text_list.append(text)

In [ ]:
for text in text_list:
  text_to_df(text)

df

,Product_name,Company_name,Chemical,Market_segmentation,CAS_No,EC_No,Hazard,UN_number
0,ANFOMUL™ 2000-LQ-(RB),Croda Inc.,Not found,Raw material,"108-31-6, 096-00-9",203-571-6,0,Not regulated as a dangerous good
1,ANFOMUL™ 2500-LQ-(RB),Croda Inc.,"Polyisobutene lactone, based polymer.",Emulsifier,"64742-54-7, 118685-28-2",,0,Not found
2,ANFOMUL S4M-LQ-(GD),Croda Inc.,Not found,Emulsifier,,,0,Not regulated as a dangerous good
3,ATMER 100-LQ-(GD),Croda Inc.,Not found,Surfactant,,,0,Not regulated as a dangerous good SAFETY DATA ...
4,ATMER™ 1013-PA-(RB),Croda Inc.,Not found,Raw material,,,0,Not regulated as a dangerous good
5,ATMER 116-LQ-(CQ),Croda Inc.,Not found,Raw material,,,0,Not regulated as a dangerous good
6,ATMER™ 190-PA-(RB),Croda Inc.,Not found,Additive,68037-49-0,,1,Not found
7,ATMER 262-LQ-(CQ),Croda Inc.,"Ethanol, 2,2'-iminobis-, N-C12-18-alkyl derivs.",Antistatic agent,71786-60-2,,1,UN 2735
8,ATMER™ 7306-PL-(RB),Croda Inc.,Not found,Antistatic agents,,,0,Not found
9,CRODAMIDE ™ 212-BE-(HU),Equus USA Inc.,Stearyl Erucamide,Lubricant,10094-45-8,,0,Not found


**Dataframe to EXCEL**

In [ ]:
df.to_excel('Output.xlsx')

**Dataframe to SQL**

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [ ]:
df.to_sql(name='product', con=engine)

60

In [ ]:
from sqlalchemy import text
with engine.connect() as conn:
    conn.execute(text("SELECT * FROM product")).fetchall()

In [ ]:
query = "SELECT * FROM product"

In [ ]:
df_filtered = pd.read_sql_query(query, con=engine)

In [ ]:
df_filtered

,index,Product_name,Company_name,Chemical,Market_segmentation,CAS_No,EC_No,Hazard,UN_number
0,0,IONPHASE RSTAT2 -PL-(FI),Croda Europe Limited,Not found,Additive,83145 -78-2,,1,Not regulated as a dangerous good
1,1,IONPHASE U1 -PL-(FI),Croda Europe Limited,Not found,Additive,83145 -78-2,,1,Not regulated as a dangerous good
2,2,KEMELIX D309-LQ-(CQ),Croda Inc.,Concentrated solution of an ethoxylated phenol...,De-emulsifier,"91-20-3, 30846-35-6",,1,Not found
3,3,PERFAD ™ FA 3340 -LQ-(GD),Croda Europe Limited,Polyester,Fuel additive,,,,Not found
4,4,PRIPOL 1022 VEG -LQ-(GD),Croda Europe Limited,Dimer acid,Raw material,,,0,Not regulated as a dangerous good
5,5,PRIFAC™ 7990-SO-(GD),Croda Inc.,Modified fatty acid,Industrial,68955-98-6,,0,Not found
6,6,SP PRIOLUBE 3970 MBAL -LQ-(GD),Croda Inc.,Ester,"Hydraulic fluids, Lubricants, greases, release...",11138-60-6,,0,Not found
7,7,PRIPLAST ™ 3192-SO-(GD),Croda Inc.,Polyester diol,Raw material,,,0,Not found
8,8,PRIPOL™ 1029-LQ-(GD),Croda Inc.,Dimer acid,Industrial,61788-89-4,,0,Not found
9,9,PRIPOL™ 1040-LQ-(GD),Croda Inc.,Not found,Industrial,68937-90-6,,0,Not found
